# [Nanoparticle number concentration measurements by multi-angle dynamic light scattering](https://doi.org/10.1007/s11051-020-04840-8)

Implementation following

    Austin, J., Minelli, C., Hamilton, D. et al. Nanoparticle number concentration measurements by multi-angle dynamic light scattering. J Nanopart Res 22, 108 (2020). https://doi.org/10.1007/s11051-020-04840-8

In [ ]:
pathToluene = "20210511/092 2021 Toluol"
pathBuffer = "20210511/093 2021 Wasser"
pathSample = "20210511/094 2021 PS-Standard 1zu1000"
refracParticle = 1.492

### Helpers

In [ ]:
# derived from '2020-07-31 DLS concentration vs. count rate'
import os
from pathlib import Path
import pandas as pd
from analyse_dls_with_contin.dlshelpers import getDLSFileData
    
def processDLSMeasurements(files, monitorDiodeRef=0):
    """Treat a set of files as measurement of the same sample,
       at different scattering angles and with repetitions, possibly.
       Returns count rate averages and std. deviations for each angle."""
    if isinstance(files, str):
        files = (files,)
    # gather all relevant data from files in a directory
    dirData = [getDLSFileData(fn) for fn in files]
    if not len(dirData):
        return
    #print(dirData)
    # create a DataFrame from that
    df = pd.DataFrame({key: [fileData[key] for fileData in dirData]
                       for key in dirData[0].keys() if key not in ("countrate", "angles")})
    monitorDiodeScale = monitorDiodeRef / df.monitorDiode.mean() if monitorDiodeRef != 0 else 1
    name = '+'.join(list(set([Path(fn).parent.name for fn in files])))
    summary = dict(name=name, timestamp=df.timestamp[0],
                   monitorDiode=df.monitorDiode.mean(),
                   monitorDiodeScale=monitorDiodeScale,
                   concentration=df.concentration[0])
    for name in ("Temperature", "Viscosity", "Refractive Index", "Wavelength"):
        for colname in df.columns:
           if colname.startswith(name):
               summary[colname] = df[colname].mean()
    print("Monitor diode reference: {:.0f}, this mon. diode: {:7.0f}, scale: {:.6f} ({})"
          .format(monitorDiodeRef, summary['monitorDiode'], monitorDiodeScale, summary['name']))
    # all sets of angles found in files in this directory, a set can have unique entries only
    dirAngles = set([tuple(fileData['angles']) for fileData in dirData])
    countRates = []
    for angles in dirAngles:
        # concatenate countrates (time series) of the same angles only (for averaging over time later)
        countRates.append(pd.concat([fileData['countrate']
                                     for fileData in dirData
                                     if tuple(fileData['angles']) == angles]))
    # count rate mean over all measurements (must be at same angles!)
    countrate = pd.concat(countRates, axis=1)*monitorDiodeScale
    print("Measurement angles:", sorted(countrate.columns.astype(int)))
    countrate.sort_index(axis=1, inplace=True)
    def seriesToDict(series, lbl):
        return {"{}{:.0f}".format(lbl, key): value for key, value in zip(series.index, series.values)}
    summary.update(seriesToDict(countrate.mean(), 'crmean'))
    summary.update(seriesToDict(countrate.std(), 'crstd'))
    return summary

def crAtAngle(summaryDict, degrees):
    return (summaryDict.get(f'crmean{degrees:.0f}', -1.),
            summaryDict.get(f'crstd{degrees:.0f}', -1.))

def crMean(summaryDict):
     return np.array([crAtAngle(summaryDict, theta)[0] for theta in getAngles(summaryDict)])

def getAngles(summaryDict):
    prefix = 'crmean'
    return sorted([int(key[len(prefix):]) for key in summaryDict.keys() if key.startswith(prefix)])

In [ ]:
from analyse_dls_with_contin.jupyter_analysis_tools.datalocations import getDataDirs, getDataFiles, printFileList

In [ ]:
import numpy as np
tolueneFiles = getDataFiles(pathToluene, include="*.ASC", exclude="_average")
printFileList(tolueneFiles)
tolueneData = processDLSMeasurements(tolueneFiles)
crTol_mean = crMean(tolueneData)
tolueneData, crTol_mean

In [ ]:
bufferFiles = getDataFiles(pathBuffer, include="*.ASC", exclude="_average")
printFileList(bufferFiles)
bufferData = processDLSMeasurements(bufferFiles, monitorDiodeRef=tolueneData['monitorDiode'])
crBuf_mean = crMean(bufferData)
bufferData, crBuf_mean

In [ ]:
particleFiles = getDataFiles(pathSample, include="*.ASC", exclude="_average")
printFileList(particleFiles)
particleData = processDLSMeasurements(particleFiles, monitorDiodeRef=tolueneData['monitorDiode'])
crTot_mean = crMean(particleData)
particleData, crTot_mean

## The differential scattering cross section using Mie theory

See https://pymiescatt.readthedocs.io/en/latest/forward.html#functions-for-single-particles

Refractive Index of
- Polypropylene, PP: 1.492 (https://www.osapublishing.org/ao/abstract.cfm?uri=ao-42-3-592)
- Polystyrene, PS: 1.593

### Using [miepython](https://miepython.readthedocs.io/en/latest/index.html)

See also https://miepython.readthedocs.io/en/latest/03_angular_scattering.html#Differential-Scattering-Cross-Section  
cites [Wiscombe, W. J. (1979). Mie Scattering Calculations. doi:10.5065/D6ZP4414](https://opensky.ucar.edu/islandora/object/technotes:232)

In [ ]:
import miepython
from analyse_dls_with_contin.jupyter_analysis_tools.utils import isList
def diffScatteringCrossSection(radius, angles, wavelen, nMedium, verbose=False):
    geometric_cross_section = np.pi * radius**2 * 1e-14 # cm**2
    if not isList(angles):
        angles = np.array([angles])
    mu = np.cos(np.radians(angles))
    m, x = refracParticle/nMedium, np.pi*2*radius/(wavelen/nMedium)
    if verbose:
        print(wavelen, 2*radius, nMedium, refracParticle/nMedium, x, geometric_cross_section)
    qext, qsca, qback, g = miepython.mie(m,x)
    return geometric_cross_section * qext * miepython.i_unpolarized(m,x,mu)

In [ ]:
# Example for testing
import numpy as np
radius = 850.
wavelen = bufferData["Wavelength [nm]"]
nMedium = bufferData["Refractive Index"]
angles = getAngles(tolueneData)
# angles = np.linspace(0,180,1000)
sigma_sca = diffScatteringCrossSection(radius, angles, wavelen, nMedium, verbose=True)

In [ ]:
# plotting Mie scattering and count rates over the angles
import matplotlib.pyplot as plt
from analyse_dls_with_contin.jupyter_analysis_tools.plotting import createFigure
createFigure(300, 2.2, quiet=True, tight_layout = {'pad': 0.05});
plt.subplot(2,3,1)
plt.plot(angles, sigma_sca, '.', color='blue')
plt.yscale('log'); plt.grid(True)
plt.title(f"Radius={radius:.0f} nm, Refractive index m={refracParticle:.2f}, $\lambda$={wavelen:.1f} nm")
plt.xlabel(r"Scattering Angle $\Theta$ (degrees)");
plt.ylabel("Diff. Scattering Cross Section (cm$^2$/sr)");

plt.subplot(2,3,2)
plt.plot(angles, sigma_sca*crTol_mean, '.', color='blue')
plt.yscale('log'); plt.grid(True)
plt.xlabel(r"Scattering Angle $\Theta$ (degrees)");
plt.ylabel("Diff. Scattering Cross Section (cm$^2$/sr)$\;\cdot\; I_{tol}$");

plt.subplot(2,3,3)
plt.plot(angles, crTol_mean, '.', color='blue')
plt.yscale('log'); plt.grid(True)
plt.title("$I_{tol}$, Intensity of Toluene")
plt.xlabel(r"Scattering Angle $\Theta$ (degrees)");

plt.subplot(2,3,4)
rayleighRatio = 1.35e-5 # in 1/cm at wavelength of 632.8 nm and 25 °C
plt.plot(angles, (crTot_mean - crBuf_mean) * rayleighRatio, '.', color='blue')
plt.yscale('log'); plt.grid(True)
plt.ylabel(r"$(I_{tot}-I_{dis}) R_{Tol}$")
plt.xlabel(r"Scattering Angle $\Theta$ (degrees)");

## Get the intensity distribution

**According to Malvern:**  
*"The first order result from a DLS experiment is an intensity distribution of particle sizes. The intensity distribution is naturally weighted according to the scattering intensity of each particle fraction or family. For biological materials or polymers the particle scattering intensity is proportional to the square of the molecular weight."*  
( https://www.chem.uci.edu/~dmitryf/manuals/Fundamentals/DLS%20terminology.pdf )

**Check:**  
[The CONTIN algorithm and its application to determine the size distribution of microgel suspensions ](https://doi.org/10.1063/1.4921686)

### Run CONTIN

In [ ]:
sizeRangeM = (1e-9, 200e-9)
continConfig = dict(
    angle=90,
    ptRangeSec=(3e-7, 1e0), fitRangeM=sizeRangeM, gridpts=200,
    transformData=True, freeBaseline=True, weighResiduals=False,
)
import copy
def updatedDict(d, key, value):
    dd = copy.copy(d)
    dd[key] = value
    return dd

In [ ]:
angles = getAngles(particleData)
continConfigs = [updatedDict(continConfig, 'angle', angle) for angle in getAngles(particleData)]
continConfigs

In [ ]:
from analyse_dls_with_contin.contin import runContinOverFiles, getContinResults
resultDirs = runContinOverFiles(particleFiles, continConfigs, nthreads=None)
resultDirs

### Rayleigh ratio
*"The Rayleigh ratio of toluene is known from the literature and is equal to 1.35·10e−5·cm−1 at 632.8 nm and 25 °C (Brar and Verma 2011)"*

In [ ]:
from analyse_dls_with_contin.jupyter_analysis_tools.distrib import area, integrate
rayleighRatio = 1.35e-5 # in 1/cm at wavelength of 632.8 nm and 25 °C
def getConcentration(continResultDir, particleData, bufferData, tolueneData, plotRangeX=None, plot=False):
    """Implements [equation 8 in Austin 2020](https://link.springer.com/content/pdf/10.1007/s11051-020-04840-8.pdf)."""
    dfDistrib, dfFit, varmap = getContinResults(continResultDir)
    theta = varmap['angle']
    I_tot, _ = crAtAngle(particleData, theta)
    I_dis, _ = crAtAngle(bufferData, theta)
    I_tol, _ = crAtAngle(tolueneData, theta)
    intensity = (I_tot - I_dis) * rayleighRatio / (I_tol)
    wavelen = bufferData["Wavelength [nm]"]
    nMedium = bufferData["Refractive Index"]
    sigma_sca = np.array([diffScatteringCrossSection(radius, theta, wavelen, nMedium)
                          for radius in dfDistrib.radius.values*1e9]).flatten()
    concentrationDistrib = dfDistrib.distrib.values * intensity / sigma_sca
    concentration = integrate(dfDistrib.radius.values, concentrationDistrib)
    if plot:
        createFigure(300, 2, quiet=True, tight_layout = {'pad': 0.05});
        plt.subplot(2,2,1); plt.grid()
        plt.errorbar(dfDistrib.radius,
                 dfDistrib.distrib * intensity, yerr=dfDistrib.err*intensity, ecolor='salmon')
        plt.title(continResultDir.name)
        plt.xlim(plotRangeX)
        plt.subplot(2,2,2); plt.grid()
        plt.plot(dfDistrib.radius.values, sigma_sca, label="Diff. scattering cross section (cm$^2$/sr)")
        plt.xlim(plotRangeX); plt.ylim(0, 1e-11); plt.legend()
        plt.subplot(2,1,2); plt.grid()
        plt.plot(dfDistrib.radius.values, concentrationDistrib,
                 label=r"concentration in $sr/cm^3$(?)=$\int${:.3g}".format(concentration));
        plt.legend(); plt.xlim(plotRangeX);
    return concentration, theta

concentrations = np.array([getConcentration(dn, particleData, bufferData, tolueneData, plotRangeX=sizeRangeM, plot=True)
                           for dn in resultDirs])

In [ ]:
createFigure(300, 3, quiet=True, tight_layout = {'pad': 0.05});
plt.subplot(1,2,1)
plt.plot(concentrations[:,1], concentrations[:,0], '.')
plt.grid(); plt.ylim(0,30e2)
plt.ylabel(r"Concentration in $1/cm^3$(?)")
plt.xlabel(r"Scattering Angle $\Theta$ (degrees)");
plt.subplot(1,2,2)
plt.plot(concentrations[:,1], concentrations[:,0], '.')
plt.grid(); plt.ylim(0,10e2)
plt.ylabel(r"Concentration in $1/cm^3$(?)")
plt.xlabel(r"Scattering Angle $\Theta$ (degrees)");
concentrations.mean(), concentrations.std()